# Optimized Budget Allocation for Maximizing Leads: A Campaign Performance Enhancement Model

---



---



In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 74.0 MB/s eta 0:00:00


In [63]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus
import pandas as pd

# Load campaign data from Excel
df = pd.read_excel("Campaign_Data_Valid.xlsx")

# Extract data
campaigns = df["Campaign Name"].tolist()
actual_leads = dict(zip(campaigns, df["Generated Accounts"].tolist()))
target_leads = dict(zip(campaigns, df["Target Accounts"].tolist()))
current_spend = dict(zip(campaigns, df["Media Spent ($)"].tolist()))
budget = dict(zip(campaigns, df["Campaign Budget ($)"].tolist()))

# Define Optimization Model
model = LpProblem("Campaign_Spend_Optimization", LpMinimize)

# Decision Variables (Optimized budget for each campaign)
x = {c: LpVariable(f"x_{c}", 0, budget[c]) for c in campaigns}  # Optimized budget
optimized_leads = {c: LpVariable(f"leads_{c}", actual_leads[c] + 1, None, cat='Integer') for c in campaigns}  # Ensure leads are strictly greater than actual leads

# Auxiliary variables for absolute deviation
deviation = {c: LpVariable(f"dev_{c}", 0) for c in campaigns}

# Objective Function: Minimize total deviation from target-based ideal budget
model += lpSum(deviation[c] for c in campaigns)

# Constraints
for c in campaigns:
    # Ideal budget calculation (scaled based on target leads)
    ideal_budget = (target_leads[c] / sum(target_leads.values())) * sum(budget.values())

    # Deviation constraints
    model += x[c] - ideal_budget <= deviation[c]
    model += ideal_budget - x[c] <= deviation[c]

    # Ensure optimized budget is within a reasonable range
    model += x[c] >= current_spend[c] + 1  # Optimized budget > actual spend
    model += x[c] <= budget[c]  # Optimized budget <= campaign budget

    # Ensure optimized leads are strictly greater than actual leads
    model += optimized_leads[c] >= actual_leads[c] + 1  # Optimized leads > actual leads
    model += optimized_leads[c] >= target_leads[c]  # Optimized leads >= target leads

    # Introduce a new variable for proportion (avoiding division)
    proportion = LpVariable(f"prop_{c}", 0)
    model += proportion * budget[c] == x[c]  # Define proportion constraint

    # Ensure optimized leads are proportional to the optimized budget
    model += optimized_leads[c] <= proportion * actual_leads[c]  # Leads proportional to budget

# Constraint: Total optimized budget should be less than or equal to total actual budget
model += lpSum(x[c] for c in campaigns) <= sum(budget.values())

# Solve the model
model.solve()

# Print Results
print("\nOptimized Campaign Allocation:")
total_optimized_budget = 0
total_actual_budget = sum(budget.values())
total_actual_leads = sum(actual_leads.values())
total_optimized_leads = 0

for c in campaigns:
    optimized_budget = max(0, x[c].varValue if x[c].varValue is not None else 0)  # Ensure non-negative values
    optimized_leads_val = max(actual_leads[c] + 1, int(optimized_leads[c].varValue if optimized_leads[c].varValue is not None else actual_leads[c] + 1))
    total_optimized_budget += optimized_budget
    total_optimized_leads += optimized_leads_val
    print(f"Campaign {c}: Optimized Budget = ${optimized_budget:.2f}, Actual Budget = ${budget[c]:.2f}, Actual Spend = ${current_spend[c]:.2f}, Actual Leads = {actual_leads[c]}, Optimized Leads = {optimized_leads_val}, Target Leads = {target_leads[c]}")

# Print Total Summary
print(f"\nTotal Summary:")
print(f"Total Actual Budget: ${total_actual_budget:.2f}")
print(f"Total Optimized Budget: ${total_optimized_budget:.2f}")
print(f"Total Actual Leads: {total_actual_leads}")
print(f"Total Optimized Leads: {total_optimized_leads}")
print(f"Status: {LpStatus[model.status]}")


Optimized Campaign Allocation:
Campaign C1: Optimized Budget = $9202.00, Actual Budget = $9202.00, Actual Spend = $6995.00, Actual Leads = 254, Optimized Leads = 255, Target Leads = 286
Campaign C2: Optimized Budget = $6831.00, Actual Budget = $6831.00, Actual Spend = $6126.00, Actual Leads = 261, Optimized Leads = 262, Target Leads = 209
Campaign C3: Optimized Budget = $9468.00, Actual Budget = $9468.00, Actual Spend = $8677.00, Actual Leads = 233, Optimized Leads = 234, Target Leads = 283
Campaign C4: Optimized Budget = $9309.98, Actual Budget = $10008.00, Actual Spend = $8651.00, Actual Leads = 117, Optimized Leads = 118, Target Leads = 276
Campaign C5: Optimized Budget = $7859.51, Actual Budget = $7920.00, Actual Spend = $5734.00, Actual Leads = 272, Optimized Leads = 273, Target Leads = 233
Campaign C6: Optimized Budget = $9264.00, Actual Budget = $11608.00, Actual Spend = $9263.00, Actual Leads = 201, Optimized Leads = 202, Target Leads = 216
Campaign C7: Optimized Budget = $870

Optimized Budget Allocation for Maximizing Leads: A Campaign Performance Enhancement Model